download de pdf usando requests lendo pdf direto do buffer da memoria

In [1]:
# file = requests.get('https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta=2019|09_Setembro|DODF%20185%2027-09-2019|&arquivo=DODF%20185%2027-09-2019%20INTEGRA.pdf')

# with open('novo_test.pdf','wb') as arqv:
#     arqv.write(file.content)

# file = requests.get('https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta=2019|09_Setembro|DODF%20185%2027-09-2019|&arquivo=DODF%20185%2027-09-2019%20INTEGRA.pdf')
# pdf = PyPDF2.PdfFileReader(io.BytesIO(file.content))
# pdf.numPages

In [2]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
import io
import re
from multiprocessing.pool import ThreadPool
import pandas as pd

In [3]:
cookies = {
    '_ga': 'GA1.4.1676642850.1569274904',
    '_gid': 'GA1.4.30087383.1569274904',
    'PHPSESSID': 'c8gp0cncq0bshqvnkq5ng3d0g7',
}

headers = {
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Origin': 'https://www.dodf.df.gov.br',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en,en-US;q=0.9,pt-BR;q=0.8,pt;q=0.7',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
    'Accept': '*/*',
    'Referer': 'https://www.dodf.df.gov.br/listar',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive',
    'Content-Length': '0',
}

In [4]:
base_url = "https://www.dodf.df.gov.br/listar"

In [6]:
params = (('dir', '/'),)
response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
years = response.json()['data'][:4]
years = list(map(lambda x:str(x),years))

all_docs = []
for year in years:
    params = (('dir', year),)
    response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
    months = response.json()['data']
    for month in months:
        params = (('dir', year+'/'+month),)
        response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
        docs = response.json()['data']
        docs = list(docs.values())
        for doc in docs:
            params = (('dir', year+'/'+month+'/'+doc),)
            response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
            all_docs.append(response.json()['data'][0])

In [15]:
#tentar fazer sem a pandas
# all_docs = [re.sub(pattern=r"(MarÃ§o)", repl="Março", string=doc) for doc in all_docs]
# all_docs = pd.DataFrame(all_docs)
# all_docs.to_csv('all_documents.csv',index=False,header=False)

In [5]:
with open('all_documents.csv','r') as file:
    all_docs = file.readlines()

In [6]:
all_docs = [re.sub(pattern=r"(\n)", repl="", string=doc) for doc in all_docs]

In [8]:
all_docs = all_docs[:5]

In [9]:
all_docs

['2019/09_Setembro/DODF 185 27-09-2019/DODF 185 27-09-2019 INTEGRA.pdf',
 '2019/09_Setembro/DODF 070 27-09-2019 EDICAO EXTRA/DODF 070 27-09-2019 EDICAO EXTRA.pdf',
 '2019/09_Setembro/DODF 184 26-09-2019/DODF 184 26-09-2019 INTEGRA.pdf',
 '2019/09_Setembro/DODF 069 25-09-2019 EDICAO EXTRA/DODF 069 25-09-2019 EDICAO EXTRA.pdf',
 '2019/09_Setembro/DODF 183 25-09-2019/DODF 183 25-09-2019 INTEGRA.pdf']

In [13]:
pool = ThreadPool(2)

In [14]:
all_data = {}
def handler(string):
    st_p, nd_p, rd_p, th_p = \
    re.sub(pattern=r"\s", repl="%20",string=string).split('/')
    url = "https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta="+st_p+"|"+nd_p+"|"+rd_p+"|&arquivo="+th_p
    pdf = requests.get(url)
    key = string.split('/')[-1]
    all_data[key] = pdf
#     with open("./docs/"+th_p,'wb') as file:
#             file.write(pdf.content)
    return all_data

In [16]:
for i in pool.map(handler, all_docs):
    print(i)

{'DODF 070 27-09-2019 EDICAO EXTRA.pdf': <Response [200]>, 'DODF 185 27-09-2019 INTEGRA.pdf': <Response [200]>, 'DODF 069 25-09-2019 EDICAO EXTRA.pdf': <Response [200]>, 'DODF 184 26-09-2019 INTEGRA.pdf': <Response [200]>, 'DODF 183 25-09-2019 INTEGRA.pdf': <Response [200]>}
{'DODF 070 27-09-2019 EDICAO EXTRA.pdf': <Response [200]>, 'DODF 185 27-09-2019 INTEGRA.pdf': <Response [200]>, 'DODF 069 25-09-2019 EDICAO EXTRA.pdf': <Response [200]>, 'DODF 184 26-09-2019 INTEGRA.pdf': <Response [200]>, 'DODF 183 25-09-2019 INTEGRA.pdf': <Response [200]>}
{'DODF 070 27-09-2019 EDICAO EXTRA.pdf': <Response [200]>, 'DODF 185 27-09-2019 INTEGRA.pdf': <Response [200]>, 'DODF 069 25-09-2019 EDICAO EXTRA.pdf': <Response [200]>, 'DODF 184 26-09-2019 INTEGRA.pdf': <Response [200]>, 'DODF 183 25-09-2019 INTEGRA.pdf': <Response [200]>}
{'DODF 070 27-09-2019 EDICAO EXTRA.pdf': <Response [200]>, 'DODF 185 27-09-2019 INTEGRA.pdf': <Response [200]>, 'DODF 069 25-09-2019 EDICAO EXTRA.pdf': <Response [200]>, 'D